# Dissecting Goldfarb-Idnani (1983)

D. Goldfarb and A. Idnani wrote a paper, __A numerically stable dual method for solving strictly convex quadratic programs__, in Mathematical Programming 27 (1983) 1-33. 

It's a little dense, and there aren't awesomely written reference implementations out there.  Part of the goal of this notebook is to 'chainsaw' through it in order to get solid guidance for a Python implementation using NumPy/SciPy, which should aid as a reference for a reviewed implementation in Fortran 90. 

## Introduction

Directly quoting, 

>   We are concerned in this paper with the strictly convex (positive definite) quadratic programming problem 
>$$ \text{min}_x \quad f(x) = a^T x + \frac{1}{2} x^T G x $$
>
>subject to 
>
>$$ s(x) \equiv C^T x - b \geq 0 $$
>
> where $x$ and $a$ are $n$-vectors. $G$ is an $n \times n$ symmetric positive definite matrix, $C$ is an $n \times m$ matrix, $b$ is an $m$-vector, and superscript $T$ denotes the transpose.  Although the vector of variables $x$ may also be subject to equality constraints
> $$ \hat{C}\hspace{0.1em}^T x - \hat{b} = 0 $$
> we shall ignore such constraints for the moment in order to simplify our presentation. 


However, we're also going to note/paraphrase the [Wikipedia article's](https://en.wikipedia.org/wiki/Quadratic_programming#Problem_formulation) notation for the formulation of the quadratic minimization problem. 

> The quadratic programming problem with $n$ variables and $m$ constraints can be formulated as follows. Given: 
> * a real-valued $n$-dim vector $\mathbf{c}$,
> * an $n \times n$-dim real symmetric matrix $Q$,
> * an $m \times n$-dim real matrix $A$, and 
> * an $m$-dim real vector $\mathbf{b}$
>
> The objective of quadratic programming is to find an $n$-dim vector $\mathbf{x}$, that will
> $$\begin{aligned} \text{minimize} \quad & f(x) = \frac{1}{2} \mathbf{x}^T Q \mathbf{x} + \mathbf{c}^T \mathbf{x} \\ \text{subject to} \quad & \quad A \mathbf{x} \preceq \mathbf{b} \end{aligned}$$


In either of these, 
* $m$ is the number of constraints
* $n$ is the number of variables
* $\mathbf{b}$ is the constraint vector
* $A \equiv C^T$ is a matrix representing the bound/constraint values for each $x_i$. 
* $Q \equiv G$ is the positive-definite matrix holding coefficients for the quadratic function. 
* $\mathbf{c} \equiv a$ is a vector containing the linear coefficients of $x$. 

Also, this student remarks that $Ax \preceq b$ means that the inequality is meant to be handled element wise. 

Quoting the paper, "In this paper we present a projection type dual algorithm for solving the \[aforementioned\] QPP".  

That is, they use the Lagrangian formulation of the big problem, and recursively solve quadratic sub-problems.  Then, iterations of solving each sub-problem is involved in solving the next. 

Quoting again, 

> "In a dual method for the strictly convex QPP one must first provide a dual feasible point, that is, a primal optimal point for some subproblem of the original problem. By relaxing all of the constraints \[in the constraint problem\], the unconstrained minimum of \[ the big quadratic problem \] is such a point.  A dual algorithm then iterates until primal feasibility (i.e. dual optimality) is achieved, all the while maintaining the primal optimality of intermediate subproblems (i.e. dual feasibility)."

Paraphrasing the remainder of the paragraph: 

This can handle the positive semi-definite case for $Q$ (solving the "dual" subproblem by a "primal method" involving the original quadratic minimization problem). 

Quoting: 

> "The important observation to make is that the origin in the
space of dual variables is always dual feasible, so that no phase 1 is needed"

In other words, by recursively/iteratively solving all these dual subproblems, we end up with a feasible point that is optimal for the original problem. 




Then, Goldfarb-Idnani talk about the remaining sections of their paper. 

* __Section 2__ "Our discussion is in terms of this problem rather than in terms of the problem dual to it... We also introduce some notation in this section along with some other preliminaries.

* __Section 3__ "The dual algorithm is given ... where its validitiy and finite termination are probed." 

* __Section 4__ "A particular numerically stable and efficient implementation of the algorithm is described..."

* __Section 5__ "...we give the results of computational tests performed on randomly generated quadratic programs...."

* __Section 6__ "...the performance of our algorirthm when used in a successive quadratic programming code is described." 

> "In both of \[Sections 5 and 6\], the computational performance of the dual algorithm is compared against that of primal algorithms.

* __Section 7__ "...Comparisons between our dual algorithm and other modified simplex type dual algorithms for quadratic programming are given..."

* __Section 8__ "...we make some remarks on alternative primal approaches and implementations of our dual algorithm and comment upon its superior behavior in degenerate and 'near' degenerate situations. 

* __Appendix__ "...in which we work through an example to illustrate the various parts of the dual algorithm." 

As implementors looking at research from the 1980's, we're going to pay particular attention to Sections 2, 3, 4, and the Appendix. 

## 2. Basic approach and preliminaries

> "The dual algorithm that is described in the next section is of the active set type.  By an _active set_ we mean a subset of the $m$ constraints in \[the constraint problem\] that are satisfied by the current estimate $x$ of the solution to the QPP \[ the big one \]". 

They use $K = \{ 1, 2, \ldots, m \}$ to denote the set of indices of the constraints of \[ the constraint problem \] and $ A \subseteq K$ to denote the indicies of the active set. 

Then, a _subproblem_ $P(J)$ is a QPP with the objective function (big quadratic) subjecti only to a subset of the constraints $J \subset K$, indexed by $J$. 

e.g., Solving $P(\emptyset)$ is the problem of finding the unconstrainted minimum of the big objective function. 

_A solution S-pair_ is $(x, A)$, where

* $x$ is a solution to a subproblem $P(J)$
* $x$ lies on some linearly independent active set of constraints $A \subseteq J$. 

If $(x, A)$ is a solution pair for a subproblem on a subset of constraints ($P(J)$), then it must also be a solution for the subproblem on the active set of constraints ($P(A)$). 

By linear independence, they mean that the normals corresponding to those constraints are linearly independent. 

Pseudo-code time!

* _Step 0_: Assume that some solution pair $(x, A)$ is given. 

> That is, start at a given minimum point of the solution space $x$ on the active set of constraints. 

* _Step 1_: Repeat until all constraints are satisfied:

    > Keep solving subproblems until every constraint $K$ has been included. 
    
    + Choose a violated constraint $p \in K \setminus A$
    
    > Choose a constraint not in the active set.  
    
    + If $P(A \cup \{p\})$ is infeasible, STOP, the QPP is infeasible. 
    
    > If trying to solve the subproblem with the new constraint is infeasible, solving the big objective function is infeasible.  That is, solving the constraint problem doesn't satisfy the pointwise inequality $Ax \preceq b$. 
    
    + Else, obtain a new S-pair $(\bar{x}, \bar{A} \cup \{p\})$ where $\bar{A} \subseteq A$ and $f(\bar{x}) > f(x)$ and set $(x, A) \leftarrow (\bar{x}, \bar{A} \cup \{p\})$.
    
    > Else, solve the next subproblem including the the chosen constraint $p$.  This gives you a new solution pair of $x$ and a new active set of constraints (note: might be subset of prior active set, but it includes the new constraint). 
    
* _Step 2_: STOP - $x$ is the optimal solution to the QPP. 

Next, they note that you can always start at $x^0 = - G^{-1} a$, an unconstrained minimum of the objective function. Then, Step 1(c) becomes

_Step 1(c')_: Determine a solution pair by solving the subproblem on the active constraints in addition to a new constraint.  Set the new solution pair to the result. 

In section 3, they describe a _dual_ algorithm to implement regular Step 1(c), which maintains dual feasibility is maintained at every point along the solution path. 

Then, from references \[18, 24\], they describe and compare a _primal-dual_ algorithm which uses the modified Step 1(c').  Preview, in section 3, their _dual_ algorithm used fewer steps. 

---

Turns out, there's a bunch of additional notation needed to describe their implementation. 

## 3. The dual algorithm



## 4. Numerically stable implementation

Take some inputs

* a positive definite symmetric Hessian matrix $G$

Their implementation will use the Cholesky factorization 

$G = L L^T$

# wat.jpg

I hit a point where my brain started leaking out of my ears a little bit, so here's where I'm sketching out what this is supposed to look like. 

The appendix in the Goldfarb-Idnani paper actually has a solid walkthrough of an example problem, so we'll go with that. 

> Nts: don't actually need to keep the following in mind. 
>
> Some reading to keep in mind:
> https://link.springer.com/article/10.1007/s00211-017-0892-8>
> 
> Let's also see if we can use the cvxopt implementation of the qp solver to any effect. 
> https://github.com/cvxopt/cvxopt/blob/f122639fc7605a04a46296eca67497b73081e617/src/python/coneprog.py#L4156



### Example

minimize $f(x) = 6 x_1 + 2(x_1^2 - x_1 x_2 + x_2^2)$

Subject to 

$$\begin{aligned}
x_1 &\geq 0 \\
x_2 &\geq 0 \\
x_1 + x_2 &\geq 2
\end{aligned}$$

In the parlance of the original term, that means we have 

* The linear coefficients of the objective function, $$ a = \begin{pmatrix}6 \\ 0\end{pmatrix} $$

* The og G matrix of quadratic coefficients for the quadratic function, $$ G = \begin{bmatrix} 4 & -2 \\ -2 & 4 \end{bmatrix} $$

For the linear inequality constraints, 

At this point, we know that a big callable function will probably look like 

```python
def ng_qpsolve(objective_mat, linear_coef_vec, constraint_mat, constraint_vec
               n_var, meq, q):
    """
        Minimize x in        f(x) = a^T x + (1/2) x^T G x
        subject to           s(x) = C^T x - b >= 0
        
        where
            - n_var                         the number of variables, 'n'

            - meq                           the number of equality constraints

            - q                             the number of total constraints

            - objective_mat     'G_{n×n}',  a positive definite symmetric matrix

            - linear_coef_vec   'a_{n×1}',  the linear coefficient vector from 
                                            the objective function above
                                            NOTE:> This gets transposed
            
            - constraint_mat    'C_{n×q}',  the row dimension 'n_var' and column 
                                            dimension 'q' constraint matrix
                                            NOTE:> This gets transposed

            - constraint_vec    'b_{q×1}',  vector of values for the q total
                                            constraints
    """

    ...

    return x
```